In [1]:
from typing import Callable

import gc
import os
import glob
import shutil
import subprocess

import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
from matplotlib.pyplot import cm

In [2]:
end_padding_time = 2                # Number of seconds to repeat for the last frame
fps = 35                            # Frame rate, 35 for Doom's default settings
cmap = cm.jet                       # Colour map for semantic segmentation
save_path = "logs/media"            # Directory to save videos
tmp_path = f"{save_path}/tmp"       # Directory to save temporary images

In [3]:
conversion_command = "ffmpeg -framerate 35 -i %s -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p %s"

In [4]:
def load_merge(path_: str, load_obs: bool=True, load_pos: bool=True, obs_transform: Callable=lambda x: x):
    obs, pos, ep_ends, weapon = [], [], [], []
    ep_offset = 0
    matching_files = glob.glob(path_)
    if len(matching_files) < 1:
        raise FileNotFoundError(f"No match for glob expression: {path_}")
    for f in tqdm(matching_files, desc="loading saved data"):
        x = np.load(f)
        if load_obs:
            obs.append(x["obs"])
            if "ep_ends" in x:
                ep_ends.append(np.asarray(x["ep_ends"], dtype=np.uint64) + ep_offset)
                ep_offset += len(obs[-1])
        if load_pos:
            pos.append(obs_transform(x["feats"]))
        if "weapon" in x:
            weapon.append(x["weapon"])
    return (np.concatenate(obs, axis=0) if load_obs else [], 
            np.concatenate(pos, axis=0) if load_pos else [],
            np.concatenate(ep_ends, axis=0) if ep_ends else [],
            np.concatenate(weapon, axis=0) if weapon else [])

In [5]:
def render_as_mp4(obs: np.ndarray, ep_ends: np.ndarray, save_name: str, scale: int=0) -> list[subprocess.CompletedProcess]:
    # We only know how many digits to reserve for temporary images at run time
    tmp_n_digits = str(len(str(obs.shape[0])))

    last_ep_end = 0
    ffmpeg_jobs = []

    try:
        if os.path.exists(tmp_path):
            shutil.rmtree(tmp_path)
        os.makedirs(tmp_path)
    except:
        print(f"Failed attempt at cleaning tmp directory, consider manual cleaning later")

    for ep_num, ep in (pbar := tqdm(enumerate(ep_ends.tolist(), 1), total=ep_ends.shape[0], 
                                    desc="rendering ep 1")): #, ncols=100)):
        frames = []
        for i in range(last_ep_end, ep):
            mapped = np.array(cmap(obs[i, 3, :, :])[:, :, :3] * 255, dtype=np.uint8)
            img = np.concatenate([obs[i, :3, :, :].transpose(1, 2, 0), mapped])
            if scale:
                img = np.repeat(np.repeat(img, repeats=scale, axis=0), repeats=scale, axis=1)
            frames.append(Image.fromarray(img))
        frames += [frames[-1]] * (fps * end_padding_time).__ceil__()
        last_ep_end = ep + 1
        pbar.set_description(f"saving ep {ep_num}")
        
        tmp_template = f"{tmp_path}/{ep_num}_%0{tmp_n_digits}d.png"
        for i, img in enumerate(frames):
            img.save(tmp_template %i)
        
        pbar.set_description(f"converting ep {ep_num}")
        mp4_path = f"{save_path}/{save_name}_ep{ep_num}.mp4"
        if os.path.exists(mp4_path):
            os.remove(mp4_path)

        cmd = conversion_command % (tmp_template, mp4_path)
        ffmpeg_jobs.append(subprocess.run(cmd, shell=True))
        
        pbar.set_description(f"rendering ep {ep_num+1}")
    
    try:
        if os.path.exists(tmp_path):
            shutil.rmtree(tmp_path)
    except:
        print(f"Failed attempt at cleaning tmp directory, consider manual cleaning later")

    return ffmpeg_jobs

In [6]:
save_name = "rtss_map1"
obs, _, ep_ends, _ = load_merge(f"logs/{save_name}_small_ss_rgb_1e-3/record_*.npz", load_pos=False)
gc.collect()
render_as_mp4(obs, ep_ends, save_name)

Loading saved data:   0%|          | 0/4 [00:00<?, ?it/s]

rendering ep 1:   0%|          | 0/20 [00:00<?, ?it/s]

[CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/1_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map1_ep1.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/2_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map1_ep2.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/3_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map1_ep3.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/4_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map1_ep4.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/5_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map1_ep5.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/6_%05d.png -c:v libx264 -crf 21 -preset fast -f

In [7]:
save_name = "rtss_map2s"
obs, _, ep_ends, _ = load_merge(f"logs/{save_name}_small_ss_rgb_1e-3/record_*.npz", load_pos=False)
gc.collect()
render_as_mp4(obs, ep_ends, save_name)

loading saved data:   0%|          | 0/4 [00:00<?, ?it/s]

rendering ep 1:   0%|          | 0/20 [00:00<?, ?it/s]

[CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/1_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map2s_ep1.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/2_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map2s_ep2.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/3_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map2s_ep3.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/4_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map2s_ep4.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/5_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map2s_ep5.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/6_%05d.png -c:v libx264 -crf 21 -preset fa

In [8]:
save_name = "rtss_map3"
obs, _, ep_ends, _ = load_merge(f"logs/{save_name}_small_ss_rgb_1e-3/record_*.npz", load_pos=False)
gc.collect()
render_as_mp4(obs, ep_ends, save_name)

loading saved data:   0%|          | 0/4 [00:00<?, ?it/s]

rendering ep 1:   0%|          | 0/20 [00:00<?, ?it/s]

[CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/1_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map3_ep1.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/2_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map3_ep2.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/3_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map3_ep3.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/4_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map3_ep4.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/5_%05d.png -c:v libx264 -crf 21 -preset fast -f mp4 -pix_fmt yuv420p logs/media/rtss_map3_ep5.mp4', returncode=0),
 CompletedProcess(args='ffmpeg -framerate 35 -i logs/media/tmp/6_%05d.png -c:v libx264 -crf 21 -preset fast -f